# Analysis

FILES

https://raw.githubusercontent.com/ScryptIQ-ai/lesson-data/refs/heads/main/dh/EEG_absence.txt
https://raw.githubusercontent.com/ScryptIQ-ai/lesson-data/refs/heads/main/dh/EEG_background.txt

PACKAGES

pandas
matplotlib
numpy
scipy

## Introduction

**Learning Objectives**
- Learn about the fourier transformation and how to apply it
- How to apply correlation analysis to time series data

The Fourier spectrum decomposes the time series into a sum of sine waves. The spectrum shows the amplitude of each sine wave component present in the signal. The coefficients are directly related to the amplitudes required to optimally fit the sum of all sine waves, in order to recreate the original data.

However, the assumption behind the Fourier Transform, is that the data are provided as an infinitely long, stationary time series. These assumptions are invalid, as the data are finite and stationarity of a biological system is rarely guaranteed. Thus, interpretation needs to be approached cautiously.

## Transform of EEG data

We import the Fourier Transform function `fft` from the library `scipy.fftpack` where it can be used to transform all columns at the same time.

In [ ]:
from scipy.fftpack import fft

To plot the results of the Fourier Transform, the following steps must be taken.

Firstly, we must obtain a Fourier spectrum for every data column. Thus, we need to define how many plots we want to have. If we take only the columns in our data, we should be able to display them all, simultaneously.

In [ ]:
data_back_fft = fft(data_back, axis=0)

Secondly, the Fourier Transform results in twice the number of complex coefficients; it produces both positive and negative frequency components, of which we only need the first (positive) half.

Lastly, the Fourier Transform outputs complex numbers. To display the 'amplitude' of each frequency, we take the absolute value of the complex numbers, using the `abs()` function.

In [ ]:
rows = data_back.shape[0]

freqs = (sr/2)*linspace(0, 1, int(rows/2))

amplitudes_back = (2.0 / rows) * abs(data_back_fft[:rows//2])

Now we have wrangled the data into a format that can be plotted using MatplotLib.

TIP BOX

Change the value of `no_win` in all the plots below to see more channels

In [ ]:
no_win = 2 # number of sensors

# sharex shares the x-axis numbers
fig, axes = subplots(figsize=(6, 5), ncols=1, nrows=no_win, sharex=True) 

names = df_back.columns[:no_win]

for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_back[:, index])
    axes[index].set_xlim(0, 8) # limit the x-axis to between 0 - 8
    axes[index].set(ylabel=f'Amplitude {names[index]}')
    axes[index].set(xlabel='Frequency (Hz)')

show()

In these two channels, we can clearly see that the main amplitude contributions lie in the low frequencies, below 2 Hz.

Let us compare the corresponding figure for the case of seizure activity:

In [ ]:
data_epil_fft = fft(data_epil, axis=0)

In [ ]:
no_win = 2 # number of sensors

fig, axes = subplots(figsize=(6, 5), ncols=1, nrows=no_win, sharex=False)

names = df_back.columns[:no_win]

amplitudes_epil = (2.0 / rows) * abs(data_epil_fft[:rows//2])

for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_epil[:, index])
    axes[index].set_xlim(0, 12)
    axes[index].set(ylabel=f'Amplitude {names[index]}')
    axes[index].set(xlabel='Frequency (Hz)')

show()

During the seizure, it is clear that the main frequency of the epileptic rhythm is between 2 and 3 Hz.

As we can see from the Fourier spectra generated above, the amplitudes are high for low frequencies; and these amplitudes tend to decrease as the frequency increases. Thus, it can sometimes be useful to see the high frequencies enhanced. This can be achieved with a logarithmic plot of the powers.

In [ ]:
no_win = 2 # number of sensors

fig, axes = subplots(figsize=(6, 6), ncols=1, nrows=no_win, sharex=True)

names = df_back.columns[:no_win]

for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_back[:, index])
    axes[index].set_xlim(0, 30)
    axes[index].set(ylabel=f'Amplitude {names[index]}')
    axes[index].set_yscale('log') # set the y-axis to natural log

axes[no_win-1].set(xlabel='Frequency (Hz)')
fig.suptitle('Logarithmic Fourier Spectra of Background EEG', fontsize=16)

show()

And for the seizure data:

In [ ]:
no_win = 2 # number of sensors

fig, axes = subplots(figsize=(6, 10), ncols=1, nrows=no_win, sharex=True)

names = df_back.columns[:no_win]

for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_epil[:, index])
    axes[index].set_xlim(0, 30)
    axes[index].set(ylabel=f'Power {names[index]}')
    axes[index].set_yscale('log')

axes[no_win-1].set(xlabel='Frequency (Hz)')
fig.suptitle('Logarithmic Fourier Spectra of Seizure EEG', fontsize=16)

show()

In the spectrum of the absence data, it is now more obvious that there are further maxima at 6, 9, 12 and perhaps 15Hz. These are integer multiples or 'harmonics' of the basic frequency at around 3Hz, which we term as the fundamental frequency.

A feature that can be used as a summary statistic, is to calculate the band power for each channel. Band power is the total power of a signal within a specific frequency range. The band power can be obtained by calculating the sum of all powers within a specified range of frequencies; this range is also referred to as the 'band'. The band power, thus, is given as a single number.

PRACTICE EXERCISE

Calculate and display the Fourier spectra of the first two channels filtered between 4 and 12 Hz for the absence seizure data.
Can you find any harmonics?

In [ ]:
# Write your code here:

## Cross-Correlation Matrix

As one example of a multivariate analysis of time series data, we can also calculate the cross-correlation matrix.

Let us calculate it for the background:

In [ ]:
from numpy import corrcoef, fill_diagonal

In [ ]:
corr_matrix_back = corrcoef(data_back, rowvar=False)

fill_diagonal(corr_matrix_back, 0)

fig, ax = subplots(figsize = (8,8))

im = ax.imshow(corr_matrix_back, cmap='coolwarm')

fig.colorbar(im, orientation='horizontal', shrink=0.68)

show()

The diagonal is set to zero. This is done to improve the visual display. If it was left set to one, the diagonal would dominate the visual impression given, even though it is trivial and uninformative.

Looking at the non-diagonal elements, we find:
- Two strongly correlated series (indices 5 and 7)
- Two strongly anti-correlated series (indices 3 and 4)
- A block of pronounced correlations (between series with indices 4 through 9)

PRACTICE EXERCISE

Calculate the correlation matrix for the seizure data and compare the correlation pattern to the one from the background data.

*The solution is given below as we discuss the comparison after*

In [ ]:
# Write your code here:

In [ ]:
corr_matrix_epil = corrcoef(data_epil, rowvar=False)

fill_diagonal(corr_matrix_epil, 0)

fig, ax = subplots(figsize = (8,8))

im = ax.imshow(corr_matrix_epil, cmap='coolwarm')

fig.colorbar(im, orientation='horizontal', shrink=0.68)

show()

We find a number of pairs of strongly correlated series, two strongly anti-correlated series (indices 3 and 4) and a block of pronounced correlations between series with indices 4 through 9.

So interestingly, while the time series changes dramatically in shape, the correlation pattern still shows some qualitative resemblance.

All results shown so far, represent the recording of the segment of 6 seconds we chose at the beginning of the lesson. The human brain produces time-dependent voltage changes 24 hours a day. Thus seeing only a few seconds provides only a partial view. The next step is therefore to investigate and demonstrate how the features found for one segment may vary over time.

## Observations

### 1. Background:

- There are irregular oscillations of all recorded brain potentials.
- Oscillations recorded at different locations above the brain, differ.
- Oscillations are not stable, but are modulated over time.
- There are different frequency components evident in each trace.

### 2. Epileptic Seizure:

- There are regular oscillations.
- Oscillations recorded at different locations are not identical but similar or at least related in terms of their shape.
- Despite some modulation, the oscillations are fairly stable over time.
- There are repetitive motifs comprising two major components throughout the recording, a sharp spike and a slow wave.

## Summary

We have quantified features of these time series data to obtain an overview of the data. For univariate features, we can use the frequency content. This approach recognises that the rows (or samples) aren't independent but are organised along the time axis. Consequently, data points along each column are correlated, and the Fourier spectrum can identify these correlations.

The Fourier spectrum assumes the data are stationary and represent a superposition of regular sine waves at different frequencies. It reveals which frequencies are present in the data and their respective amplitudes. The spectrum is obtained through the Fourier Transform, which decomposes a signal into its constituent frequencies, enabling analysis of individual frequency components and inferences about periodic characteristics.

For bivariate features, we can use the cross-correlation matrix.